In [1]:
import json
import pandas as pd


with open('data/wbm/40344463.json', 'r') as f:
    data = json.load(f)
df = pd.read_csv('data/wbm/44225498.csv')

In [2]:
df.shape

(256963, 18)

In [3]:
idxs = [17 + i * 20 for i in range(10000)]
idx2id = {idx: data['material_id'][str(idx)] for idx in idxs}

In [4]:
from tqdm.auto import tqdm

structs = {idx2id[idx]: data['computed_structure_entry'][str(idx)] for idx in tqdm(idxs)}
energy = {idx2id[idx]: df[df['material_id'] == idx2id[idx]]['e_form_per_atom_wbm'].item() for idx in tqdm(idxs)}
band_gap = {idx2id[idx]: df[df['material_id'] == idx2id[idx]]['bandgap_pbe'].item() for idx in tqdm(idxs)}

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
from pymatgen.io.cif import CifWriter
from pymatgen.core import Structure

In [6]:
res = {
    'cif': [str(CifWriter(Structure.from_dict(structs[idx2id[idx]]['structure']))) for idx in idxs],
    'formation_energy_per_atom': [energy[idx2id[idx]] for idx in idxs],
    'band_gap': [band_gap[idx2id[idx]] for idx in idxs]
}

In [7]:
df1 = pd.DataFrame.from_dict(res)
df1.head()

,cif,formation_energy_per_atom,band_gap
0,# generated using pymatgen\ndata_CeAg2Hg\n_sym...,-0.162,0.0
1,# generated using pymatgen\ndata_InAg2\n_symme...,-0.019,0.0
2,# generated using pymatgen\ndata_SmAg2\n_symme...,-0.289,0.0
3,# generated using pymatgen\ndata_TaAg2\n_symme...,0.281,0.0
4,# generated using pymatgen\ndata_Ag3Bi\n_symme...,0.045,0.0


In [8]:
df1.to_csv('data/wbm/val.csv.gz')